将whole、institution、individual、momentum、reverse文件转换为矩阵形式，并存为mat文件

In [1]:
import json
import math
import datetime
import numpy as np
import pickle
import scipy.io as scio
import pymysql

conn = pymysql.connect(
    host = '219.224.169.45',
    user = 'lizimeng',
    password = 'codegeass',
    db = 'market',
    charset = 'utf8'
)
with conn.cursor() as cursor:
    cursor.execute("select day from trade_day where day between '20140101' and '20161231' order by day asc")
    days = [x for x, in cursor.fetchall()][1:]
conn.close()

cols = [
    'am', 'agm', 'alm', 'apm', 'aum', 'bm', 'bbm', 'bum', 'cm', 'cfm', 'csm', 'cum', 'cym', 'erm', 'fbm', 'fgm', 'fum', 'hcm', 
    'im', 'jm', 'jdm', 'jmm', 'jrm', 'lm', 'lrm', 'mm', 'mam', 'mem', 'nim', 'oim', 'pm', 'pbm', 'pmm', 'ppm', 'rbm', 'rim', 
    'rmm', 'rom', 'rsm', 'rum', 'sfm', 'smm', 'snm', 'srm', 'tam', 'tcm', 'vm', 'whm', 'wrm', 'wsm', 'ym', 'zcm', 'znm', 'ap', 
    'agp', 'alp', 'app', 'aup', 'bp', 'bbp', 'bup', 'cp', 'cfp', 'csp', 'cup', 'cyp', 'erp', 'fbp', 'fgp', 'fup', 'hcp', 'ip', 
    'jp', 'jdp', 'jmp', 'jrp', 'lp', 'lrp', 'mp', 'map', 'mep', 'nip', 'oip', 'pp', 'pbp', 'pmp', 'ppp', 'rbp', 'rip', 'rmp', 
    'rop', 'rsp', 'rup', 'sfp', 'smp', 'snp', 'srp', 'tap', 'tcp', 'vp', 'whp', 'wrp', 'wsp', 'yp', 'zcp', 'znp', 'fund'
]

def triple2matrix(trip):
    mat = np.zeros((len(cols), len(cols)))
    for t in trip:
        i = cols.index(t[0])
        j = cols.index(t[1])
        mat[i,j] = t[2]
    return mat

def str2date(s):
    '''
    yyyy-mm-dd -> datetime.date(yyyy,mm,dd)
    '''
    year = int(s[:4])
    month = int(s[5:7])
    day = int(s[8:])
    return datetime.date(year,month,day)


In [3]:
# day
for fname in ['whole', 'institution', 'individual', 'momentum', 'reverse']:
    tensor = np.zeros((107,107,732))
    with open(fname, 'rb') as f:
        i = 0
        for cf in pickle.load(f):
            cf = triple2matrix(cf)
            tensor[:,:,i] += cf
            i += 1
    with open('%s_day.data'%fname,'wb') as f:
        pickle.dump(tensor, f)


In [2]:
# month
for fname in ['whole', 'institution', 'individual', 'momentum', 'reverse']:
    with open(fname,'rb') as f:
        tensor = np.zeros((107,107,36))
        i = 0
        for cf in [triple2matrix(x) for x in pickle.load(f)]:
            d = days[i]
            tensor[:,:,(d.year-2014)*12+d.month-1] += cf
            i += 1
    with open('%s_month.data'%fname,'wb') as f:
        pickle.dump(tensor, f)


In [ ]:
# day matrix
for fname in ['whole', 'institution', 'individual', 'momentum', 'reverse']:
    with open(fname, 'rb') as f:
        mat = np.zeros((107*107,732))
        i = 0
        for cf in pickle.load(f):
            cf = triple2matrix(cf)
            mat[:,i] += cf.reshape(-1)
            i += 1
    scio.savemat('M_' + fname + '_day.mat', {'M':mat})

In [ ]:
# month matrix
for fname in ['whole', 'institution', 'individual', 'momentum', 'reverse']:
    with open(fname, 'rb') as f:
        mat = np.zeros((107*107,36))
        i = 0
        for cf in pickle.load(f):
            d = days[i]
            cf = triple2matrix(cf)
            mat[:,(d.year-2014)*12+d.month-1] += cf.reshape(-1)
            i += 1
    scio.savemat('M_' + fname + '_month.mat', {'M':mat})

In [ ]:
# month normal
for fname in ['whole', 'institution', 'individual', 'momentum', 'reverse']:
    with open(fname,'rb') as f:
        tensor = np.zeros((107,107,36))
        i = 0
        for cf in [triple2matrix(x) for x in pickle.load(f)]:
            d = days[i]
            tensor[:,:,(d.year-2014)*12+d.month-1] += cf
            i += 1
        for i in range(36):
            total = np.sum(tensor[:,:,i])
            for j in range(107):
                for k in range(107):
                    tensor[j,k,i] = tensor[j,k,i] / total
    scio.savemat(fname+'_month_norm.mat', {'M':tensor})

In [ ]:
# month log
for fname in ['whole', 'institution', 'individual', 'momentum', 'reverse']:
    with open(fname,'rb') as f:
        tensor = np.zeros((107,107,36))
        i = 0
        for cf in [triple2matrix(x) for x in pickle.load(f)]:
            d = days[i]
            tensor[:,:,(d.year-2014)*12+d.month-1] += cf
            i += 1
        for i in range(36):
            tensor[:,:,i] = np.log(tensor[:,:,i]+1)
    scio.savemat(fname+'_month_log.mat', {'M':tensor})

In [ ]:
a = np.ones((3,4))

In [ ]:
a

In [ ]:
a[:,1]